In [ ]:
# Importations essentielles
import numpy as np
import pandas as pd

# Visualisation
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import xgboost as xgb
from catboost import CatBoostRegressor

# Suppression des warnings inutiles
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)


print("Jupyter Notebook est bien configuré ! 🚀")

In [ ]:
# Charger le dataset
df = pd.read_csv("train.csv") 

# Aperçu des données
print(df.head())

# Informations générales sur les colonnes
print(df.info())

# Statistiques sur les variables numériques
print(df.describe())

In [ ]:
# Vérifier combien de colonnes ont des valeurs manquantes
missing_values = df.isnull().sum()
missing_values = missing_values[missing_values > 0]  # Afficher uniquement les colonnes concernées

print("Colonnes avec valeurs manquantes :")
print(missing_values.sort_values(ascending=False))

# Visualiser les valeurs manquantes avec seaborn

plt.figure(figsize=(10, 6))
sns.heatmap(df.isnull(), cmap="viridis", cbar=False, yticklabels=False)
plt.title("Carte des valeurs manquantes")
plt.show()

In [ ]:
# Suppression des colonnes avec trop de valeurs manquantes
df = df.drop(columns=["PoolQC", "MiscFeature", "Alley", "Fence"])

# Remplacement des NaN pour les variables catégoriques (remplace NaN par "None")
cols_cat = ["GarageType", "GarageFinish", "GarageQual", "GarageCond",
            "BsmtQual", "BsmtCond", "BsmtExposure", "BsmtFinType1", "BsmtFinType2",
            "MasVnrType"]
df[cols_cat] = df[cols_cat].fillna("None")

# Remplacement des NaN pour les variables numériques (remplace NaN par la médiane)
cols_num = ["LotFrontage", "MasVnrArea", "GarageYrBlt"]
for col in cols_num:
    df[col] = df[col].fillna(df[col].median())

# Remplacement des NaN pour Electrical par la valeur la plus fréquente
df["Electrical"] = df["Electrical"].fillna(df["Electrical"].mode()[0])

# Remplacement des NaN pour FireplaceQu (qualité de la cheminée)
df["FireplaceQu"] = df["FireplaceQu"].fillna("None")

# Vérifier qu'il ne reste plus de valeurs manquantes
print("Valeurs manquantes après nettoyage :")
print(df.isnull().sum().sum())  # Doit afficher 0 si tout est bien nettoyé

In [ ]:
# Remplacement des NaN pour FireplaceQu (qualité de la cheminée)
df["FireplaceQu"] = df["FireplaceQu"].fillna("None")

print(df.isnull().sum().sum())  # Doit afficher 0 si tout est bien nettoyé

In [ ]:
# Sélectionner uniquement les colonnes numériques
df_numeric = df.select_dtypes(include=["number"])

# Calculer la matrice de corrélation uniquement sur ces colonnes
correlation_matrix = df_numeric.corr()

# Trier les variables les plus corrélées avec SalePrice
correlation_with_price = correlation_matrix["SalePrice"].sort_values(ascending=False)

# Afficher les 15 variables les plus corrélées avec SalePrice
print("Top 15 des variables les plus corrélées avec SalePrice :")
print(correlation_with_price.head(15))

# Visualisation de la heatmap de corrélation

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, cmap="coolwarm", annot=False, fmt=".2f")
plt.title("Matrice de corrélation des variables numériques")
plt.show()

In [ ]:
cat_cols = ["MSZoning", "Neighborhood", "BldgType", "HouseStyle",
            "RoofStyle", "Heating", "GarageType", "SaleCondition"]

# Compter le nombre de catégories uniques pour chaque colonne
for col in cat_cols:
    print(f"{col} : {df[col].nunique()} catégories uniques")

In [ ]:
# Liste des colonnes catégoriques à encoder
cat_cols = ["MSZoning", "Neighborhood", "BldgType", "HouseStyle",
            "RoofStyle", "Heating", "GarageType", "SaleCondition"]

# Appliquer l'encodage One-Hot et supprimer la première colonne de chaque catégorie (drop_first=True)
df_encoded = pd.get_dummies(df, columns=cat_cols, drop_first=True)

# Vérifier le nouveau nombre de colonnes
print("Nombre de colonnes après encodage :", df_encoded.shape[1])

# Afficher un aperçu des nouvelles colonnes
print(df_encoded.head())

In [ ]:
# Sélectionner uniquement les colonnes numériques après encodage
df_numeric = df_encoded.select_dtypes(include=["number"])

# Calculer la matrice de corrélation
correlation_matrix = df_numeric.corr()

# Trier les variables les plus corrélées avec SalePrice
correlation_with_price = correlation_matrix["SalePrice"].sort_values(ascending=False)

# Afficher les 15 variables les plus corrélées avec SalePrice
print("Top 15 des variables les plus corrélées avec SalePrice :")
print(correlation_with_price.head(15))

# Visualisation de la heatmap de corrélation
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, cmap="coolwarm", annot=False, fmt=".2f")
plt.title("Matrice de corrélation des variables après encodage")
plt.show()

In [ ]:
A = correlation_matrix

print(A)

In [ ]:
# Sélection des variables les plus pertinentes
features = ["OverallQual", "GrLivArea", "GarageCars", "GarageArea",
            "TotalBsmtSF", "1stFlrSF", "FullBath", "TotRmsAbvGrd",
            "YearBuilt", "YearRemodAdd", "MasVnrArea", "Fireplaces",
            "GarageYrBlt", "BsmtFinSF1"]

# Création du dataset final avec les variables sélectionnées
X = df_encoded[features]  # Variables explicatives
y = df_encoded["SalePrice"]  # Variable cible

# Division des données en entraînement (80%) et test (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Taille de l'ensemble d'entraînement :", X_train.shape)
print("Taille de l'ensemble de test :", X_test.shape)

In [ ]:
# Initialiser et entraîner le modèle
model = LinearRegression()
model.fit(X_train, y_train)

# Prédictions sur l'ensemble de test
y_pred = model.predict(X_test)

# Évaluer le modèle
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
r2 = r2_score(y_test, y_pred)

# Afficher les résultats
print(f"Mean Absolute Error (MAE) : {mae}")
print(f"Mean Squared Error (MSE) : {mse}")
print(f"Root Mean Squared Error (RMSE) : {rmse}")
print(f"R² Score : {r2}")

In [ ]:
# Initialiser le modèle Random Forest
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Entraîner le modèle sur l'ensemble d'entraînement
rf_model.fit(X_train, y_train)

# Prédictions sur l'ensemble de test
y_pred_rf = rf_model.predict(X_test)

# Évaluer le modèle
mae_rf = mean_absolute_error(y_test, y_pred_rf)
mse_rf = mean_squared_error(y_test, y_pred_rf)
rmse_rf = mse_rf ** 0.5
r2_rf = r2_score(y_test, y_pred_rf)

# Afficher les résultats
print(f"Random Forest - Mean Absolute Error (MAE) : {mae_rf}")
print(f"Random Forest - Mean Squared Error (MSE) : {mse_rf}")
print(f"Random Forest - Root Mean Squared Error (RMSE) : {rmse_rf}")
print(f"Random Forest - R² Score : {r2_rf}")

In [ ]:
# Récupérer l'importance des variables
importances = rf_model.feature_importances_

# Créer un DataFrame pour les afficher proprement
feature_importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': importances})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# Afficher les 10 variables les plus importantes
plt.figure(figsize=(10, 6))
sns.barplot(x=feature_importance_df.Importance[:10], y=feature_importance_df.Feature[:10], palette="viridis")
plt.xlabel("Importance")
plt.ylabel("Feature")
plt.title("Top 10 des variables les plus importantes")
plt.show()

# Afficher le classement complet
print(feature_importance_df)

In [ ]:
# Définir la grille de paramètres à tester
param_grid = {
    'n_estimators': [100, 200, 300],  # Nombre d'arbres
    'max_depth': [None, 10, 20],  # Profondeur max des arbres
    'min_samples_split': [2, 5, 10],  # Nombre min d’échantillons pour diviser un nœud
    'min_samples_leaf': [1, 2, 4]  # Nombre min d’échantillons dans une feuille
}

# Initialiser le modèle
rf = RandomForestRegressor(random_state=42)

# GridSearchCV pour tester toutes les combinaisons
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

# Meilleurs paramètres trouvés
print("Meilleurs paramètres :", grid_search.best_params_)

# Tester avec ces paramètres optimisés
best_rf = RandomForestRegressor(**grid_search.best_params_, random_state=42)
best_rf.fit(X_train, y_train)
y_pred_best_rf = best_rf.predict(X_test)

# Évaluer le modèle optimisé
r2_best_rf = r2_score(y_test, y_pred_best_rf)
print("R² Score après optimisation :", r2_best_rf)

In [ ]:
# Prédictions avec le modèle optimisé
y_pred_optimized = best_rf.predict(X_test)

# Évaluation des erreurs
mae_optimized = mean_absolute_error(y_test, y_pred_optimized)
mse_optimized = mean_squared_error(y_test, y_pred_optimized)
rmse_optimized = mse_optimized ** 0.5

# Afficher les résultats
print(f"Optimized Random Forest - Mean Absolute Error (MAE) : {mae_optimized}")
print(f"Optimized Random Forest - Mean Squared Error (MSE) : {mse_optimized}")
print(f"Optimized Random Forest - Root Mean Squared Error (RMSE) : {rmse_optimized}")

In [ ]:
# Initialiser le modèle XGBoost
xgb_model = XGBRegressor(n_estimators=300, learning_rate=0.05, max_depth=6, random_state=42)

# Entraîner le modèle
xgb_model.fit(X_train, y_train)

# Prédictions
y_pred_xgb = xgb_model.predict(X_test)

# Évaluation XGBoost
mae_xgb = mean_absolute_error(y_test, y_pred_xgb)
mse_xgb = mean_squared_error(y_test, y_pred_xgb)
rmse_xgb = mse_xgb ** 0.5
r2_xgb = r2_score(y_test, y_pred_xgb)

# Afficher les résultats
print(f"XGBoost - Mean Absolute Error (MAE) : {mae_xgb}")
print(f"XGBoost - Mean Squared Error (MSE) : {mse_xgb}")
print(f"XGBoost - Root Mean Squared Error (RMSE) : {rmse_xgb}")
print(f"XGBoost - R² Score : {r2_xgb}")

In [ ]:
# Définir une grille d'hyperparamètres à tester
param_grid = {
    'n_estimators': [200, 300, 400],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 6, 9]
}

# Création du modèle XGBoost
xgb = XGBRegressor(random_state=42)

# GridSearchCV pour tester les combinaisons
grid_search_xgb = GridSearchCV(xgb, param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search_xgb.fit(X_train, y_train)

# Afficher les meilleurs paramètres
print("Meilleurs paramètres XGBoost :", grid_search_xgb.best_params_)

# Tester avec ces paramètres optimisés
best_xgb = XGBRegressor(**grid_search_xgb.best_params_, random_state=42)
best_xgb.fit(X_train, y_train)
y_pred_best_xgb = best_xgb.predict(X_test)

# Évaluer le modèle optimisé
r2_best_xgb = r2_score(y_test, y_pred_best_xgb)
print("R² Score après optimisation de XGBoost :", r2_best_xgb)

In [ ]:
# Comparer prédictions et vraies valeurs
errors = np.abs(y_test - y_pred_best_xgb)

# Afficher les 10 pires erreurs
worst_predictions = pd.DataFrame({"Actual": y_test, "Predicted": y_pred_best_xgb, "Error": errors})
worst_predictions = worst_predictions.sort_values(by="Error", ascending=False)
print("🔍 Top 10 des pires erreurs :")
print(worst_predictions.head(10))

In [ ]:
plt.figure(figsize=(8,6))
plt.scatter(y_test, y_pred_best_xgb, alpha=0.5)
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], '--', color='red')  # Ligne idéale y = x
plt.xlabel("Prix réel ($)")
plt.ylabel("Prix prédit ($)")
plt.title("🔍 Comparaison des prédictions vs prix réels")
plt.show()

In [ ]:
joblib.dump(best_xgb, "xgboost_model.pkl")
print("✅ Modèle XGBoost sauvegardé sous 'xgboost_model.pkl'")

In [ ]:
print("Maisons à plus de 500K$ :", sum(y_train > 500000))

In [ ]:
# Transformation log du prix
y_train_log = np.log(y_train)
y_test_log = np.log(y_test)

# Réentraîner XGBoost avec les prix transformés
xgb_log = XGBRegressor(n_estimators=300, learning_rate=0.05, max_depth=3, random_state=42)
xgb_log.fit(X_train, y_train_log)
y_pred_log = xgb_log.predict(X_test)

# Reconvertir les prédictions avec exp()
y_pred_final = np.exp(y_pred_log)

# Recalculer les erreurs
mae_log = mean_absolute_error(y_test, y_pred_final)
r2_log = r2_score(y_test, y_pred_final)

print(f"✅ MAE après transformation log : {mae_log}")
print(f"✅ R² Score après transformation log : {r2_log}")

In [ ]:
# Extraire les maisons à plus de 500K$
expensive_houses = X_train[y_train > 500000]
expensive_prices = y_train[y_train > 500000]

# Ajouter ces maisons plusieurs fois dans l'entraînement
X_train_balanced = pd.concat([X_train] + [expensive_houses] * 10, axis=0)
y_train_balanced = pd.concat([y_train] + [expensive_prices] * 10, axis=0)

# Réentraîner XGBoost avec ces nouvelles données
xgb_balanced = XGBRegressor(n_estimators=300, learning_rate=0.05, max_depth=3, random_state=42)
xgb_balanced.fit(X_train_balanced, y_train_balanced)
y_pred_balanced = xgb_balanced.predict(X_test)

# Évaluer
r2_balanced = r2_score(y_test, y_pred_balanced)
print(f"✅ R² Score après augmentation des maisons chères : {r2_balanced}")

In [ ]:
# Initialiser le modèle
catboost_model = CatBoostRegressor(iterations=300, depth=6, learning_rate=0.05, random_seed=42, verbose=0)

# Entraîner le modèle
catboost_model.fit(X_train, y_train)

# Prédictions
y_pred_catboost = catboost_model.predict(X_test)

# Évaluation
r2_catboost = r2_score(y_test, y_pred_catboost)
print(f"✅ R² Score avec CatBoost : {r2_catboost}")

In [ ]:
joblib.dump(catboost_model, "catboost_best_model.pkl")
print("✅ Modèle CatBoost sauvegardé sous 'catboost_best_model.pkl'")